In [23]:
import os
os.chdir("/Users/postgres/Desktop/NLP/Text-Summarizer/src")
os.getcwd()

'/Users/postgres/Desktop/NLP/Text-Summarizer/src'

In [24]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

from TextSummarizer.constants import *
from TextSummarizer.utils.common import read_yaml, create_directories

In [25]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name=config.tokenizer_name,
        )

        return data_transformation_config

In [26]:
from TextSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_metric, load_from_disk



class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_name)
    
    #the main function needed: in textSummaizer.ipynb: tokenize 
    def convert_examples_to_features(self, example_batch)-> bool:
        #could add the max_length to self.config? 
        #padding is set to default False
        input_encodings = self.tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )

        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length = 128, truncation = True )

        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    
    #save the encoded data to file samsum_dataset
    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_encoded = dataset_samsum.map(self.convert_examples_to_features, batched=True)
        dataset_samsum_encoded.save_to_disk(os.path.join(self.config.root_dir, 'samsum_dataset'))

In [27]:
os.chdir("/Users/postgres/Desktop/NLP/Text-Summarizer")
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config = data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2023-09-20 22:07:05,361] INFO: common: yaml file: config/config.yaml loaded successfully
[2023-09-20 22:07:05,363] INFO: common: yaml file: params.yaml loaded successfully
[2023-09-20 22:07:05,364] INFO: common: created directory at: artifacts
[2023-09-20 22:07:05,368] INFO: common: created directory at: artifacts/data_transformation


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]/Users/postgres/opt/anaconda3/envs/textS/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:3660: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 105932.46 examples/s]
